In [9]:
import pandas as pd
import gensim

In [10]:
df = pd.read_csv('data.csv')

In [11]:
df

,Asana,Benefits
0,PADOTTHANASANA,This asana strengthens\nthe abdominal muscles ...
1,PARVATASANA,This pose strengthens the nerves and muscles i...
2,ARDHA TITALI ASANA,This is an excellent \npreparatory practice fo...
3,GATYATMAK MERU \nVAKRASANA,This asana removes stiffness \nof the back and...
4,SIDEWAYS VIEWING,Sideways viewing relaxes the \ntension of the ...
...,...,...
321,kapilasana,"1. Helps in stretching legs, hamstrings, arms,..."
322,omkarasana,It relieves the pain of hands and feet( It is ...
323,kashyapawsana,1. This asana provides a deep muscular massage...
324,bhunamanasana,1. Bhunamanasana stretches and improves the fl...


In [12]:
#shape of the data frame
df.shape

(326, 2)

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop = stopwords.words('english')
#  1st step for removing stop words is to use the library and remove stop words, this will make
# sure that some basic stop words and numerical values is been removed from the Benefits column.

# 2nd step is that there might be stop words present in upper case for example "This" is a stop words
# which was not removed in 1st step so I converted the Benefits column to lower case. Moreover we can only
# lowercase any sentence or entire column when that column is free of any numerical value.

# 3rd then further removing stopwords, in this way we get the whole benefits column free from stop words


from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.preprocessing import strip_non_alphanum
from gensim.parsing.preprocessing import strip_numeric
from gensim.parsing.preprocessing import strip_multiple_whitespaces
df['Benefits'] = df['Benefits'].apply(str.lower)
df['Benefits']=df.Benefits.apply(remove_stopwords)
df['Benefits']=df.Benefits.apply(strip_non_alphanum)
df['Benefits']=df.Benefits.apply(strip_numeric)
df['Benefits']=df.Benefits.apply(strip_multiple_whitespaces)
df['Asana']=df.Asana.apply(strip_multiple_whitespaces)
df['Asana'] = df['Asana'].apply(str.lower)
df

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Asana,Benefits
0,padotthanasana,asana strengthens abdominal muscles massages o...
1,parvatasana,pose strengthens nerves muscles limbs back hel...
2,ardha titali asana,excellent preparatory practice loosening knee ...
3,gatyatmak meru vakrasana,asana removes stiffness increases flexibility ...
4,sideways viewing,sideways viewing relaxes tension muscles strai...
...,...,...
321,kapilasana,helps stretching legs hamstrings arms chest b...
322,omkarasana,relieves pain hands feet helpful rheumatism gi...
323,kashyapawsana,asana provides deep muscular massage abdomen ...
324,bhunamanasana,bhunamanasana stretches improves flexibility ...


In [15]:
# this will create the list of each benefit rows
benefits =df['Benefits'].apply(gensim.utils.simple_preprocess)
print(benefits)

0      [asana, strengthens, abdominal, muscles, massa...
1      [pose, strengthens, nerves, muscles, limbs, ba...
2      [excellent, preparatory, practice, loosening, ...
3      [asana, removes, stiffness, increases, flexibi...
4      [sideways, viewing, relaxes, tension, muscles,...
                             ...                        
321    [helps, stretching, legs, hamstrings, arms, ch...
322    [relieves, pain, hands, feet, helpful, rheumat...
323    [asana, provides, deep, muscular, massage, abd...
324    [bhunamanasana, stretches, improves, flexibili...
325    [stretches, strengthens, lengthens, mandalasan...
Name: Benefits, Length: 326, dtype: object


In [16]:
# prints the words present in the benefit row 1
benefits[0]

['asana',
 'strengthens',
 'abdominal',
 'muscles',
 'massages',
 'organs',
 'strengthens',
 'digestive',
 'system',
 'lower',
 'back',
 'pelvic',
 'perineal',
 'muscles',
 'helps',
 'correct',
 'prolapse']

## *Implemented Word2Vec with following features:*
* window size = 5
* minimum word cound for which the model can input row wise data = 2
* required cpu  threads to train the model = 4
* size of the required vector embedding = 50

In [18]:
model = gensim.models.Word2Vec(
    window=5,
    min_count=2,
    workers=4,
    vector_size=50,  # Use vector_size instead of size
)


## *Building vocabulary of unique words present in the entire benefit column*

In [19]:

model.build_vocab(benefits, progress_per=5)
# vocab_len = len(model.wv)
# print(vocab_len)

In [20]:
model.train(benefits, total_examples=model.corpus_count, epochs=2000)

(22225305, 29294000)

## *We can test our model for any words suppose say sciatica we will get the similar words in benefits*

In [21]:
#when we find the similar words for a disease in
#Benefits section we also see that we get the names of asanas also.
#like for this word we get similar words like chakrasana, marjariasana, shalabhasana which indicates
#that since we have taken these words from benefits section, it means that for this particular diseaes
#most prpbable asanas can be chakrasana, marjariasana, shalabhasana etc.  to cure it.
model.wv.most_similar("sciatica", topn= 100)

#print(type(model.wv.most_similar("sciatica", topn= 100)))

[('stiff', 0.43023085594177246),
 ('mild', 0.4106746315956116),
 ('pains', 0.3945704400539398),
 ('spondylitis', 0.3912590742111206),
 ('backache', 0.37916889786720276),
 ('benefit', 0.3762171268463135),
 ('time', 0.3761589229106903),
 ('rid', 0.3617485463619232),
 ('treatment', 0.34696540236473083),
 ('problem', 0.3319668769836426),
 ('sitting', 0.32827436923980713),
 ('slipped', 0.3192024230957031),
 ('on', 0.3149697482585907),
 ('squat', 0.3034071624279022),
 ('stress', 0.30278488993644714),
 ('vajra', 0.3003588020801544),
 ('relief', 0.2965806722640991),
 ('marjariasana', 0.2934306263923645),
 ('preparation', 0.2929081916809082),
 ('sciatic', 0.29035598039627075),
 ('region', 0.2881268560886383),
 ('advantage', 0.2812809646129608),
 ('easily', 0.27942073345184326),
 ('utkatasana', 0.27870771288871765),
 ('lumbago', 0.2748037576675415),
 ('drooping', 0.27432748675346375),
 ('spinal', 0.27355873584747314),
 ('disc', 0.26906776428222656),
 ('poorna', 0.26629549264907837),
 ('inside', 

## *Printing one of the benefits column word's vector*

In [ ]:
# suppose we want to know the vector embedding of a word 'pain' so we have printed the vector embedding of size 50
import numpy as np
print((model.wv.get_vector('pain')))

In [22]:
# counts total number of rows in datasets being trained
model.corpus_count

326

In [25]:
from gensim.models import Word2Vec

words = list(model.wv.key_to_index.keys())


In [ ]:
# printing unique words from the vocabulary list.
print(words)

In [26]:
# here we have the length of unique words

print(len(words))

1330


## *Created an empty dictionary at first to store the unique words as key words  along with its vector embeddings.*

In [27]:
dict_of_word_embeddings = dict({})
for i in words:
    dict_of_word_embeddings[i] = model.wv[i]

In [ ]:
# on printing the dictionary, we get the following result.
print(dict_of_word_embeddings)

## *Exporting dictionary data into separate excel sheet/ csv file*.

In [28]:
Unique_words = dict_of_word_embeddings.keys()
word_vectors  = dict_of_word_embeddings.values()
# print(asanas, word_vectors)
d = {'Unique_words' : Unique_words , 'Word_Vectors' : word_vectors}
dataframe = pd.DataFrame(data = d)
dataframe

,Unique_words,Word_Vectors
0,muscles,"[-2.5864224, 2.1722589, 0.620521, 2.7694068, -..."
1,body,"[-1.1209418, 1.3465433, -1.5965929, 2.1820705,..."
2,pose,"[-1.5228001, 0.07352058, -0.6421016, 1.1455548..."
3,helps,"[0.861907, -1.9077177, -0.3151574, 1.2105575, ..."
4,organs,"[-3.0509617, -1.8573385, -2.1302586, 3.6100235..."
...,...,...
1325,butterfly,"[2.8196752, -0.49041247, 5.4974675, 0.3384651,..."
1326,titli,"[-1.5461555, -0.8928119, 3.1140275, 2.3617275,..."
1327,tolasana,"[3.5816953, -1.8628008, -6.645945, 2.7023067, ..."
1328,improvement,"[4.926644, -0.10064105, 2.376137, -2.1765952, ..."


## *We have made list of unique asanas as after merging excel sheets there were repeated asanas*

In [29]:
asanas = list(df['Asana'])
# total asanas present ( with repetition)
print(len(asanas))
asana = []


for x in asanas:
  if x not in asana:
    asana.append(x)
# total number of unique asanas
print(len(asana))
# list of unique asanas
print(asana)

326
293
['padotthanasana', 'parvatasana', 'ardha titali asana', 'gatyatmak meru vakrasana', 'sideways viewing', 'makarasana', 'padmasana', 'vajrasana', 'ardha chandrasana', 'yogamudrasana', 'bhujangasana', 'saithalyasana', 'bhu namanasana', 'sarvangasana', 'natarajasana', 'poorna bhujangasana', 'koormasana', 'poorna shalabhasana', 'poorna dhanurasana', 'bandha hasta utthanasana ', 'shava udarakarshanasana ', 'chakki chalanasana ', 'kashtha takshanasana ', 'vayu nishkasana', 'ushtrasana', 'samakonasana ', 'matsyasana', 'kandharasana', ' setu asana ', 'paschimottanasana', 'meru akarshanasana', 'pada hastasana', 'seetkari pranayama', 'jalandhara bandha', 'tadagi mudra', 'maha vedha mudra', 'shashankasana', 'janu chakra', 'poorna titali asana', 'manibandha chakra', 'skandha chakra', 'greeva sanchalana', 'padachakrasana', 'pada sanchalanasana', 'supta pawanmuktasana', 'jhulana lurhakanasana', 'supta udarakarshanasana', 'naukasana', 'rajju karshanasana', 'nauka sanchalanasana', 'namaskarasan

## *On-hot Encodded the asanas words and created a dictionary to store the asana word with its word embedding*

In [30]:
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(asana)
#print(integer_encoded)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)

### One hot encoding
onehot_encoder = OneHotEncoder(sparse=False)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

# onehot_encoded

asan_dict={}

for i in range(len(asana)):
  asan_dict[asana[i]] = onehot_encoded[i]

print(asan_dict)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


{'padotthanasana': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
      

In [31]:
from tensorflow import keras
from tensorflow.keras.layers import Dense

In [32]:
# printing the benefit words of one row.
print(benefits[0])

['asana', 'strengthens', 'abdominal', 'muscles', 'massages', 'organs', 'strengthens', 'digestive', 'system', 'lower', 'back', 'pelvic', 'perineal', 'muscles', 'helps', 'correct', 'prolapse']


## *Created a Tupple associating the benefits words of each row with the respective asana. In order to bring the associativity between benefits words and asana words*

In [33]:
pair=[]

i=0
a=len(asana)
for x in benefits:
  if(i<a):
    target=asana[i]
    for y in x:
      if(y not in words):
        continue
      pair.append((y,target))
  i+=1
print(pair)




[('asana', 'padotthanasana'), ('strengthens', 'padotthanasana'), ('abdominal', 'padotthanasana'), ('muscles', 'padotthanasana'), ('massages', 'padotthanasana'), ('organs', 'padotthanasana'), ('strengthens', 'padotthanasana'), ('digestive', 'padotthanasana'), ('system', 'padotthanasana'), ('lower', 'padotthanasana'), ('back', 'padotthanasana'), ('pelvic', 'padotthanasana'), ('muscles', 'padotthanasana'), ('helps', 'padotthanasana'), ('correct', 'padotthanasana'), ('prolapse', 'padotthanasana'), ('pose', 'parvatasana'), ('strengthens', 'parvatasana'), ('nerves', 'parvatasana'), ('muscles', 'parvatasana'), ('limbs', 'parvatasana'), ('back', 'parvatasana'), ('helps', 'parvatasana'), ('increase', 'parvatasana'), ('height', 'parvatasana'), ('stretching', 'parvatasana'), ('muscles', 'parvatasana'), ('ligaments', 'parvatasana'), ('enabling', 'parvatasana'), ('growing', 'parvatasana'), ('bones', 'parvatasana'), ('grow', 'parvatasana'), ('longer', 'parvatasana'), ('circulation', 'parvatasana'), 

## *Making a 2-D array of context words(benefit words) and Target words(asana words) by numpy stack*

In [35]:
import numpy as np
contexts=[dict_of_word_embeddings[context] for context,target in pair]
contexts=np.vstack(contexts)
# shape of the context words matrix
contexts.shape


(12085, 50)

In [36]:
targets=[asan_dict[target] for context,target in pair]
targets=np.vstack(targets)
# shape of the target words matrix

targets.shape

(12085, 293)

## *Implemented **Artificial Neural Network**. with the help of tensorfow's keras functional API with the following features:*

* Number of input layer = 1
* Size of input layer = 50 unit
* Number of dense layer = 1
* size of dense layer = 1000 units
* Number of output layer = 1
* size of output unit = 293
* activation function = softmax
* loss = categorical_crossentropy
* optimizer = adam
* number of epotchs = 100

In [37]:
from tensorflow import keras
from tensorflow.keras.layers import Dense

network_input = keras.Input(shape=contexts.shape[1], name='input_layer')
                                                                                    # Create a hidden layer for the network; store under 'hidden_layer'
hidden_layer1 = Dense(units=1000, activation='sigmoid', name='hidden_layer1')(network_input)

                                                                                            # Create an output layer for the network; store under 'output_layer'
output_layer = Dense(units=targets.shape[1], activation='softmax', name='output_layer')(hidden_layer1)

                                                                          # Create a Keras Model; store under 'embedding_model'
embedding_model = keras.Model(inputs=network_input, outputs=output_layer)

                                                          # Compile the model for training; define loss function
embedding_model.compile(loss='categorical_crossentropy',optimizer = 'adam', metrics = ['accuracy'])

                                                          # Print out a summary of the model
embedding_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 50)]              0         
                                                                 
 hidden_layer1 (Dense)       (None, 1000)              51000     
                                                                 
 output_layer (Dense)        (None, 293)               293293    
                                                                 
Total params: 344293 (1.31 MB)
Trainable params: 344293 (1.31 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [38]:
# Fit a model to the data
embedding_model.fit(x=contexts,   # inputs
                    y=targets,   # outputs
                    batch_size=1024,  # how many pairs of words processed simultaneously
                    epochs=100,   # how many times we loop through the whole data
                    verbose=1   # do not print training status
                   )



Epoch 1/100
12/12 [==============================] - 2s 7ms/step - loss: 5.4721 - accuracy: 0.0172
Epoch 2/100
12/12 [==============================] - 0s 6ms/step - loss: 5.1424 - accuracy: 0.0401
Epoch 3/100
12/12 [==============================] - 0s 6ms/step - loss: 4.9355 - accuracy: 0.0601
Epoch 4/100
12/12 [==============================] - 0s 6ms/step - loss: 4.7753 - accuracy: 0.0775
Epoch 5/100
12/12 [==============================] - 0s 5ms/step - loss: 4.6489 - accuracy: 0.0871
Epoch 6/100
12/12 [==============================] - 0s 5ms/step - loss: 4.5462 - accuracy: 0.0936
Epoch 7/100
12/12 [==============================] - 0s 6ms/step - loss: 4.4624 - accuracy: 0.0994
Epoch 8/100
12/12 [==============================] - 0s 5ms/step - loss: 4.3892 - accuracy: 0.1025
Epoch 9/100
12/12 [==============================] - 0s 5ms/step - loss: 4.3253 - accuracy: 0.1072
Epoch 10/100
12/12 [==============================] - 0s 5ms/step - loss: 4.2755 - accuracy: 0.1088
Epoch 11/

## *Function which inputs the user details and suggests user with the most recommended asanas*

In [39]:
from collections import Counter
from IPython.display import clear_output

def magic():
  predicted_asanas = []
  user_input_words= []
  final_predicted_asanas = []
  number_in_words = ['first','second', 'third', 'fourth']
  for i in range(4):
    user_input_words.append(input(f"Enter {number_in_words[i]} benefit word:  "))
  for i in user_input_words:
    if i in dict_of_word_embeddings:

      input_array = np.expand_dims(dict_of_word_embeddings[i], axis=0)
      prediction = embedding_model.predict(input_array)
      flatten_pred = prediction.flatten()
      result_indices = flatten_pred.argsort()[-10:][::-1]

      for result in result_indices:
        predicted_asanas.append(asana[result])


  counter_found = Counter(predicted_asanas)
  final_predicted_asanas_with_freq = counter_found.most_common(7)
  # print(final_predicted_asanas_with_freq)

  for yoga, freq in final_predicted_asanas_with_freq:
    final_predicted_asanas.append(yoga)

  print(final_predicted_asanas)
  choice=input("Clear output: Y/N ")
  if choice=='Y':
   clear_output()


magic()

Enter first benefit word:  backpain
Enter second benefit word:  headache
Enter third benefit word:  ache
Enter fourth benefit word:  head
1/1 [==============================] - 0s 18ms/step
['parivritti janu sirshasana', 'pada hastasana', ' utthita hasta padangusthasana', 'ardha padma halasana', 'muktasana', ' dhanurakarshanasana', 'chakki chalanasana']
Clear output: Y/N y
